In [21]:
import os
import numpy as np
import glob
import random
import shutil

In [22]:
ISA = 'x86'
input_folder = f'dataset/graph_adj_matrix_all/{ISA}'
output_folder = f"dataset/similar_func_pairs/{ISA}"
PAIR_NUM=50000

In [23]:
def func_pair_gen(input_folder, output_folder):
    # Create output directories
    os.makedirs(os.path.join(output_folder, 'pos'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'neg'), exist_ok=True)

    # Get all CSV files
    all_files = glob.glob(f'{input_folder}/*.csv')

    # Group files by everything except optimization level
    file_groups = {}
    for file_path in all_files:
        filename = os.path.basename(file_path)
        parts = filename.split('@')
        key = '@'.join(parts[:-1])  # Everything except optimization level
        if key not in file_groups:
            file_groups[key] = []
        file_groups[key].append(file_path)

    # Generate similar pairs
    similar_pairs = []
    for group in file_groups.values():
        if len(group) > 1:
            similar_pairs.extend([(a, b) for a in group for b in group if a != b])

    # Generate unsimilar pairs
    all_files_set = set(all_files)
    unsimilar_pairs = []
    while len(unsimilar_pairs) < PAIR_NUM:
        a, b = random.sample(all_files_set, 2)
        if a.split('@')[:-1] != b.split('@')[:-1]:
            unsimilar_pairs.append((a, b))

    # Shuffle and limit to 50000 pairs each
    random.shuffle(similar_pairs)
    similar_pairs = similar_pairs[:PAIR_NUM]
    random.shuffle(unsimilar_pairs)
    unsimilar_pairs = unsimilar_pairs[:PAIR_NUM]

    # Write pairs to output folders
    for i, (a, b) in enumerate(similar_pairs):
        shutil.copy(a, os.path.join(output_folder, 'pos', f'{i}$a${os.path.basename(a)}'))
        shutil.copy(b, os.path.join(output_folder, 'pos', f'{i}$b${os.path.basename(b)}'))

    for i, (a, b) in enumerate(unsimilar_pairs):
        shutil.copy(a, os.path.join(output_folder, 'neg', f'{i}$a${os.path.basename(a)}'))
        shutil.copy(b, os.path.join(output_folder, 'neg', f'{i}$b${os.path.basename(b)}'))

In [24]:
def main():
    func_pair_gen(input_folder, output_folder)
if __name__ == '__main__':
    main()                     

/tmp/ipykernel_2627622/1539748675.py:29: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  a, b = random.sample(all_files_set, 2)
